In [ ]:
'''
Steps:
    1) Divide imput image into patches
    2) Flatten the patches to be handles as tokens
    3) Add classification token at the beginning
    4) Feed tokens into the encoder
    5) Number of output of the encoder is equal to the size of the input
    6) 
'''

In [2]:
import torch
import torch.nn as nn 
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
# Patch the input image - (Image Size / Patch Size) * (Image Size / Patch Size) 
# Patch size = the NxW size of each patch
# Sliding window technique

image_size = 256
patch_size = 16 
num_patches = (image_size // patch_size) * (image_size // patch_size)

image = np.random.rand(image_size, image_size, 3) # Example random image

def slidingWindow():
    x_start = 0
    # CONTINUE HERE!